In [2]:
import urllib, yaml
import numpy as np
import pandas as pd
import sqlalchemy

In [2]:
df_gbm = pd.read_parquet("C:/Users/bruno.gonzalez/Downloads/predmex/220126forecast_MEX_gbm.parquet")

In [3]:
df_las = pd.read_parquet("C:/Users/bruno.gonzalez/Downloads/predmex/220126forecast_MEX_lasso.parquet")

In [4]:
df_xgb = pd.read_parquet("C:/Users/bruno.gonzalez/Downloads/predmex/220126forecast_MEX_xgboost.parquet")

In [5]:
df = pd.concat([df_gbm, df_las, df_gbm])

In [6]:
df.insert(loc=0,column="Country", value='Mexico')

In [7]:
df.head()

,Country,date,ship_to_party,material,model_type,feature_forecasted,window_prediction,forecast
0,Mexico,2019-09-01,AJEMEX SA DE CV,38420903CE,gbm,3p_sales_qty_total_mt,1,7.144119
1,Mexico,2018-11-01,AJEMEX SA DE CV,Not assigned,gbm,3p_sales_qty_total_mt,1,7.144119
2,Mexico,2019-01-01,AJEMEX SA DE CV,Not assigned,gbm,3p_sales_qty_total_mt,1,7.144119
3,Mexico,2019-03-01,AJEMEX SA DE CV,Not assigned,gbm,3p_sales_qty_total_mt,1,7.144119
4,Mexico,2019-05-01,AJEMEX SA DE CV,Not assigned,gbm,3p_sales_qty_total_mt,1,7.144119


In [8]:
df.dtypes

Country                  object
date                  period[D]
ship_to_party            object
material                 object
model_type               object
feature_forecasted       object
window_prediction         int64
forecast                float64
dtype: object

In [9]:
df = df.rename(columns={
    'date': 'File Date',
    'window_prediction': 'Forecast Window'})

In [10]:
df_piv = pd.pivot_table(df,
               values='forecast',
               columns='feature_forecasted',
               aggfunc=np.sum,
               index=['Country','File Date', 'ship_to_party','material', 'model_type', 'Forecast Window']).reset_index()

In [11]:
df_piv.head()

feature_forecasted,Country,File Date,ship_to_party,material,model_type,Forecast Window,3p_sales_qty_total_mt,n3p_net_revenue,new_cogs,operating_income
0,Mexico,2018-01-01,PROVEEDORA LA PERLA SA DE CV,06420103CE,gbm,1,14.288238,9356.830494,7473.038306,2682.463222
1,Mexico,2018-01-01,PROVEEDORA LA PERLA SA DE CV,06420103CE,gbm,2,16.640435,11744.716027,8501.395281,2933.295580
2,Mexico,2018-01-01,PROVEEDORA LA PERLA SA DE CV,06420103CE,gbm,3,20.553535,13262.050354,10183.032034,3045.670859
3,Mexico,2018-01-01,PROVEEDORA LA PERLA SA DE CV,06420103CE,gbm,4,21.972371,14210.221417,10019.871305,3555.812772
4,Mexico,2018-01-01,PROVEEDORA LA PERLA SA DE CV,06420103CE,gbm,5,22.852189,15125.678656,11180.685435,3688.558770


In [7]:
with open("credentialsdb.yml") as stream:
    cred = yaml.safe_load(stream)

In [8]:
cred['DRIVER']

'{ODBC Driver 17 for SQL Server}'

In [9]:
params = urllib.parse.quote_plus(f"DRIVER={cred['DRIVER']};"
                                 f"SERVER={cred['SERVER']};"
                                 f"DATABASE={cred['DATABASE']};"
                                 f"UID={cred['UID']};"
                                 f"PWD={cred['PWD']}")



In [10]:
engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

In [16]:
# engine = sqlalchemy.create_engine(
# f"mssql+pyodbc://{cred['UID']}:{cred['PWD']}@{cred['SERVER']}/{cred['DATABASE']}"
# )

In [17]:
# df = pd.DataFrame({'a':[1,2,3], 'b':[4,5,6]})
# df.to_sql('test', con=engine, schema='clean', if_exists='append')

In [18]:
df_piv['File Date'] = df_piv['File Date'].astype(str)

In [19]:
df_piv.shape

(1684272, 10)

In [20]:
df_piv[['File Date','Forecast Window','model_type','ship_to_party','material','3p_sales_qty_total_mt','n3p_net_revenue','new_cogs', 'operating_income']].\
    to_sql('pl_data_mex_predicted2', con=engine, schema='dbo', if_exists='append', chunksize=10000, index_label=False)

In [20]:
df_piv[['File Date','Forecast Window','model_type','ship_to_party','material','3p_sales_qty_total_mt','n3p_net_revenue','new_cogs', 'operating_income']].dtypes

feature_forecasted
File Date                period[D]
Forecast Window              int64
model_type                  object
ship_to_party               object
material                    object
3p_sales_qty_total_mt      float64
n3p_net_revenue            float64
new_cogs                   float64
operating_income           float64
dtype: object

In [21]:
df_piv['File Date'].astype(str)

0          2018-01-01
1          2018-01-01
2          2018-01-01
3          2018-01-01
4          2018-01-01
              ...    
1684267    2021-12-01
1684268    2021-12-01
1684269    2021-12-01
1684270    2021-12-01
1684271    2021-12-01
Name: File Date, Length: 1684272, dtype: object

In [14]:
df_comp = pd.read_csv("./d_component.csv")

In [15]:
df_comp

,component_ac,component_pred,component_pred2
0,3P Sales Qty Total MT,3p_sales_qty_total_mt,3P Sales Qty Total MT - Predicted
1,Gross Revenue USD,Gross Revenue USD,Gross Revenue USD
2,N3p Net Revenue,n3p_net_revenue,N3P Net Revenue - Predicted
3,New COGS,new_cogs,New COGS - Predicted
4,Gross Profit,Gross Profit,Gross Profit
5,Operating Income,operating_income,Operating Income - Predicted
6,Net Income,Net Income,Net Income
7,Total Net Income,Total Net Income,Total Net Income


In [16]:
df_comp.to_sql('d_component', con=engine, schema='clean', if_exists='append', chunksize=10000, index_label=False)

In [5]:
df_mc = pd.read_csv("./MonteCarlo.csv")

In [12]:
df_mc.head()

,File Date,Country,model_type,forecast_window,metric_name,predicted
0,20200101,CAN,Monte Carlo,0,3P Sales Qty Total MT - Predicted,71156.74384
1,20200201,CAN,Monte Carlo,0,3P Sales Qty Total MT - Predicted,65614.89107
2,20200301,CAN,Monte Carlo,0,3P Sales Qty Total MT - Predicted,72083.96678
3,20200401,CAN,Monte Carlo,0,3P Sales Qty Total MT - Predicted,75227.02365
4,20200501,CAN,Monte Carlo,0,3P Sales Qty Total MT - Predicted,77830.01218


In [13]:
df_mc.to_sql('pl_data_all_predicted', con=engine, schema='dbo', if_exists='append', chunksize=10000, index=False)